## Import Package and Load Config

In [11]:
from modules.models import RRDB_Model
from modules.utils import (load_yaml, set_memory_growth, imresize_np,
                           tensor2img, rgb2ycbcr, create_lr_hr_pair,
                           calculate_psnr, calculate_ssim)
import numpy as np
import tensorflow as tf
import os
import ipywidgets as widgets	# 控件库
from IPython.display import display	# 显示控件的方法
text = widgets.Text()
display(text)
cfg_path="./configs/esrgan.yaml"
def print_value(sender):
    if os.path.isfile(sender.value):
        cfg_path=sender.value

text.on_submit(print_value)
set_memory_growth()


Text(value='')

In [12]:
cfg = load_yaml(cfg_path)

## Load Model

In [13]:
model = RRDB_Model(None, cfg['ch_size'], cfg['network_G'])
checkpoint_dir = './checkpoints/' + cfg['sub_name']
checkpoint = tf.train.Checkpoint(model=model)
if tf.train.latest_checkpoint(checkpoint_dir):
    checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
    print("[*] load ckpt from {}.".format(
            tf.train.latest_checkpoint(checkpoint_dir)))


Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7fa858defd60> and <keras.layers.core.tf_op_layer.TFOpLambda object at 0x7fa851a219d0>).



Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7fa858defd60> and <keras.layers.core.tf_op_layer.TFOpLambda object at 0x7fa851a219d0>).



Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7fa851a21280> and <keras.layers.core.tf_op_layer.SlicingOpLambda object at 0x7fa851a21820>).



Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7fa851a21280> and <keras.layers.core.tf_op_layer.SlicingOpLambda object at 0x7fa851a21820>).



Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7fa889c4d940> and <keras.layers.core.tf_op_layer.TFOpLambda object at 0x7fa858defe20>).



Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7fa889c4d940> and <keras.layers.core.tf_op_layer.TFOpLambda object at 0x7fa858defe20>).



Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7fa859aef250> and <keras.layers.core.tf_op_layer.TFOpLambda object at 0x7fa858def2b0>).



Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7fa859aef250> and <keras.layers.core.tf_op_layer.TFOpLambda object at 0x7fa858def2b0>).


[*] load ckpt from ./checkpoints/esrgan/ckpt-580.


## Save to SavedModel Format

In [14]:
tf.saved_model.save(
    model, 'saved_model', #signatures=model.signatures['serving_default']#, options=None
)

INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


## Convert SavedModel To Onnx

In [15]:
!pip install -U tf2onnx
!python -m tf2onnx.convert --saved-model saved_model --opset 13 --output esrgan.onnx

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
/opt/conda/lib/python3.8/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-04-04 08:42:32,297 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2022-04-04 08:42:43,284 - INFO - Signatures found in model: [serving_default].
2022-04-04 08:42:43,284 - WARNING - '--signature_def' not specified, using first signature: serving_default
2022-04-04 08:42:43,286 - INFO - Output names: ['conv_last']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2022-04-04 08:42:46,107 - WARNING - From /opt/conda/lib/python3.8/site-packages/tf2onnx/tf_loader.py:706: extract_sub_graph (from tensorflow.python.framework.graph_ut